In [1]:
import sys
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt

In [4]:
#-------------------------------------------------------------------------------------------------------------------------------
# Title: Solving the ODE for eta and phi, then plotting the growth rate against wavenumber
#-------------------------------------------------------------------------------------------------------------------------------
# Contents: 
#-------------------------------------------------------------------------------------------------------------------------------

# Section 0.0: List of all the profiles so that we can copy and paste to switch between them

# Section 1.0: Define a function that will integrate through the domain and return phi_{n+1} 

# Section 2.0: Define a function that will calculate P and Q in d2phi/dy2 + P(y) dphi/dy + Q(y) phi = 0
# Section 2.1: Allocate memory to the variables and define the constants in the ODE
# Section 2.2: Calculate P and Q in the ODE for eta
# Section 2.3: Calculate P and Q in the ODE for v'

# Section 3.0: Define an ODE solver that will use the previous functions to solve the ODE
# Section 3.1: Set up the grid in y, allocate memory and define the constants
# Section 3.2: Find P and Q for c_0 (first choice of wave speed) and then integrate to find phi with boundary conditions
# Section 3.3: Iterate towards a solution for the ODE over all the wave speed increments
# Section 3.4: Check for convergence and prepare for the next loop

# Section 4.0: Plots of the results



In [5]:
#------------------------------------------------------------------------------------------------------------------------------
# Section 0.0 : List of all the profiles so that we can copy and paste to switch between them
#------------------------------------------------------------------------------------------------------------------------------

#--n = 0-----------------------------------------------------------------------------------------------------------------------

# H[iy]     = A0*np.exp(-B[iy])
# dHdy[iy]  = -(A0/2.0)*C*y[iy]*np.exp(-B[iy])
# U[iy]     = (g*A0/c)*np.exp(-B[iy]) 
# dUdy[iy]  = U[iy]*0.5*y[iy]*np.sqrt(C)
# HQ[iy]    = beta*(1/C)*y[iy] + (g*A2 / 2*c)*C*y[iy]*np.exp(-B[iy]) 
# dHQdy[iy] = beta - (g*A0/4*c)*(C**3)*((y[iy]**2)-2)*np.exp(-B[iy])

#--n = 2------------------------------------------------------------------------------------------------------------------------

# H[iy]     = A2*((y[iy]**2)-1)*np.exp(-B[iy])
# dHdy[iy]  = -(A2/2.0)*C*y[iy]*((y[iy]**2)-5)*np.exp(-B[iy])
# U[iy]     = -(g*A2/c)*((y[iy]**2)-5)*np.exp(-B[iy])
# dUdy[iy]  = - (g*A2/(2*c))*C*y[iy]*np.exp(-B[iy])*((y[iy]**2)+9) 
# HQ[iy]    = beta*(1/C)*y[iy] + (g*A2/(2*c))*C*y[iy]*np.exp(-B[iy])*((y[iy]**2)+9) 
# dHQdy[iy] = beta + (g*A2/2*c)*C*(2*C*(y[iy]**2)*np.exp(-B[iy])+((y[iy]**2)+9)*(-0.5*C*((y[iy]**2)-2)*np.exp(-B[iy])))

#--Cubic------------------------------------------------------------------------------------------------------------------------

# H[iy]     = (y[iy]**3)*np.exp(-B[iy])
# dHdy[iy]  = 3*(y[iy]**2)*C*np.exp(-B[iy])-0.5*C*(y[iy]**4)*np.exp(-B[iy])
# U[iy]     = (g/c)*y[iy]*np.exp(-B[iy])*((y[iy]**2)-6)
# dUdy[iy]  =  (g/c)*(-0.5*C*((y[iy]**2)-2)*((y[iy]**2)-6)*np.exp(-B[iy])+2*C*(y[iy]**2)*np.exp(-B[iy]))
# HQ[iy]    = beta*(1/C)*y[iy] - (g/c)*(-0.5*C*((y[iy]**2)-2)*((y[iy]**2)-6)*np.exp(-B[iy])+2*C*(y[iy]**2)*np.exp(-B[iy]))
# dHQdy[iy] = beta - (g/c)*C*((-0.5*C*y[iy]*np.exp(-B[iy]))*(-0.5*((y[iy]**2) -2)*((y[iy]**2) -6)+2*(y[iy]**2)) + np.exp(-B[iy])*(-2*C*(y[iy]**3)+12*y[iy]*C))

#--Quintic----------------------------------------------------------------------------------------------------------------------

# H[iy]     = (y[iy]**5)*np.exp(-B[iy])
# dHdy[iy]  = -0.5*C*(y[iy]**4)*np.exp(-B[iy])*((y[iy]**2)-10)
# U[iy]     = (g/c)*(y[iy]**3)*np.exp(-B[iy])*((y[iy]**2)-10)
# dUdy[iy]  = (g/c)*(-(1/2)*C*((y[iy]**2))*((y[iy]**2)-6)*((y[iy]**2)-10)*np.exp(-B[iy])+2*C*(y[iy]**4)*np.exp(-B[iy]))
# HQ[iy]    = beta*(1/C)*y[iy] - (g/c)*(-(1/2)*C*((y[iy]**2))*((y[iy]**2)-6)*((y[iy]**2)-10)*np.exp(-B[iy])+2*C*(y[iy]**4)*np.exp(-B[iy]))
# dHQdy[iy] = beta - (g/c)*C*(-0.5*C*y[iy]*np.exp(-B[iy])*(-0.5*(y[iy]**2)*((y[iy]**2) -6)*((y[iy]**2) -10)+2*(y[iy]**4))+np.exp(-B[iy])*(6*C*(y[iy]**5)-56*C*(y[iy]**3)+120*C*y[iy]))

#--Sinusoidal-------------------------------------------------------------------------------------------------------------------

# m         = 1
# A0        = A1 = A2 = A3 = A4 = - 1.0
# A         = [ A0 , A1 , A2 , A3 , A4 ]
# U[iy]     = A[m]*np.sin(m*np.pi*y[iy] / L)   # L = no. of grid points
# H[iy]     = -(A2*(beta) / g)*((L/m*np.pi)**2)*((np.sin(m*np.pi*y[iy]/L)-(m*np.pi/L)*y[iy]*np.cos(m*np.pi/L)))
# dUdy[iy]  = A[m]*(m*np.pi/L)*np.cos(m*np.pi*y[iy]/L)
# dHdy[iy]  = (-f[iy]/g)*U[iy]
# HQ[iy]    = f[iy] - dUdy[iy]
# dHQdy[iy] =  beta + A[m]*((m*np.pi/L)**2)*np.sin(m*np.pi*y[iy]/L) 

In [34]:
#-------------------------------------------------------------------------------------------------------------------------------
# Section 1.0 : Define a function that will integrate through the domain and return phi_{n+1}  
#-------------------------------------------------------------------------------------------------------------------------------
def INT_REES(phi_nm1 , phi_n,  P_n , Q_n , y_nm1 , y_n , y_np1):      
  import sys
  import numpy as np
  import math
  import cmath
  import matplotlib.pyplot as plt

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# phi_nm1 = phi_{n-1} , phi_n = phi_{n} , P_n = P_{n} , Q_n = Q_{n} , y_nm1 = y_{n-1} , y_n = y_{n} , y_np1 = y_{n+1}
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Finite Difference Scheme

# It is assumed that the grid spacing is uniform

# Using a centered finite difference scheme 
# phi = phi_n                                                          
# d/dy phi = (phi_np1-phi_nm1) / 2(y_np1-y_n)                                
# d^{2}/dy^{2} phi = (phi_np1-2*phi_n+phi_nm1) / (y_np1-y_n)^{2}

# phi_np1 [ 1.0/(y_np1 - y_n)^2 + P_n/(y_np1 - y_nm1) ] +  [ (- 2 phi_n + phi_nm1 )/(y_np1 - y_n)^2  -  P_n phi_nm1/(y_np1 - y_nm1) + Q_n phi_n ] = 0

  dy      = y_np1 - y_n 
  denom   = 1.0 / (dy*dy) + (P_n/(y_np1 - ynm_1))               
  num2    = - (P_n * phi_nm1 / (y_np1 - y_nm1)) - Q_n * phi_n   
  num1    = (-2.0 * phi_n + phi_nm1) / (dy*dy) 
  phi_np1 = - (num1 + num2) / denom

  return phi_np1

In [35]:
#------------------------------------------------------------------------------------------------------------------------------
# Section 2.0 : Define a function that will calculate P and Q in d2phi/dy2 + P(y) dphi/dy + Q(y) phi = 0
#------------------------------------------------------------------------------------------------------------------------------

# Calculating P and Q
def P_Q_Rees(y, c_n):   # y is an array of np.zeros, and k is the wavenumber
  import sys
  import numpy as np
  import math
  import cmath
  import matplotlib.pyplot as plt

#------------------------------------------------------------------------------------------------------------------------------
# Section 2.1 : Allocate memory to the variables and define the constants in the ODE
#------------------------------------------------------------------------------------------------------------------------------
  
  len_y = len(y)  # y=np.zeros( no. of levels + 2 , dtype =np.complex ) , see section 3.1
  
  # Allocate memory for the numpy arrays   
  P      = np.zeros( len_y, dtype =  np.complex )     
  Q      = np.zeros( len_y, dtype =  np.complex )     
  f      = np.zeros( len_y, dtype =  np.complex )
  B      = np.zeros( len_y, dtype =  np.complex )
  H      = np.zeros( len_y, dtype =  np.complex )
  dHdy   = np.zeros( len_y, dtype =  np.complex )
  U      = np.zeros( len_y, dtype =  np.complex )
  dUdy   = np.zeros( len_y, dtype =  np.complex )
  HQ     = np.zeros( len_y, dtype =  np.complex )
  dHQdy  = np.zeros( len_y, dtype =  np.complex )
  D      = np.zeros( len_y, dtype =  np.complex )
  dDdy   = np.zeros( len_y, dtype =  np.complex )
  F      = np.zeros( len_y, dtype =  np.complex )
  dFdy   = np.zeros( len_y, dtype =  np.complex )
  d2Udy2 = np.zeros( len_y, dtype =  np.complex )

  # Define the constants
  beta   = 2.2*(10**-11)       
  g      = 0.0334              # Reduced gravity
  H0     = 60                  # Average thermocline depth (see Tanaka+Hibiya)
  A0     = -1.0                # Constant for n=0
  A2     = -1.0                # Constant for n=2
  c      = np.sqrt(g*H0)      
  C      = np.sqrt((2*beta/c))  
  k      = 1.0
  #y     = C*x                 # Non-dimensional y

#------------------------------------------------------------------------------------------------------------------------------
# Section 2.2 : Calculate P and Q in the ODE for eta
#------------------------------------------------------------------------------------------------------------------------------
  for iy in range ( len_y ) :
    f[iy]     = beta*y[iy]
    B[iy]     = (y[iy]**2)/4.0
    H[iy]     = A0 * np.exp(-B[iy])                                                     
    dHdy[iy]  = - (A0 / 2.0) * C * y[iy] * np.exp(-B[iy])                               
    U[iy]     = (g * A0 / c) * np.exp(-B[iy])                                           
    dUdy[iy]  = U[iy] * 0.5 * y[iy] * C
#   d2Udy2[iy] = ((g*A0*beta)/(2*c)) * ((y[iy]**2)-2)*np.exp(-B[iy])
    HQ[iy]    = beta * (1 / C) * y[iy] + (g * A0 / 2 * c) * C * y[iy] * np.exp(-B[iy])  
    dHQdy[iy] = beta - (g*A0/4*c)*(C**3)*((y[iy]**2)-2)*np.exp(-B[iy])
    D[iy]     = (k**2) * ((U[iy] - c_n)**2) - f[iy] * HQ[iy]
    dDdy[iy]  = 2 * (k**2) * dUdy[iy] * (U[iy] - c_n) - beta * HQ[iy] - f[iy] * dHQdy[iy] # 2 * (k**2) * dUdy[iy] * (U[iy] - c) - 2 * (beta**2) * y[iy] + beta * dUdy[iy] + beta * y[iy] * d2Udy2[iy]
    F[iy]     = D[iy] / (g * H[iy])
    dFdy[iy]  = 1.0 / (g * H[iy]) * dDdy[iy] - D[iy] / g * H[iy] * (dHdy[iy])
    P[iy]     = (-1.0 / F[iy])*dFdy[iy]
    Q[iy]     = ((F[iy] - (k**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_n))
    
#------------------------------------------------------------------------------------------------------------------------------
# Section 2.3 : Calculate P and Q in the ODE for v'
#------------------------------------------------------------------------------------------------------------------------------
# Comment out F, dFdy, P, Q in Section 2.2 and then uncomment G, dGdy, P, Q here

#   G[iy]     = (k**2)*((U[iy]-cs)**2 - g*H[iy])
#   dGdy[iy]  = (k**2)*(2*dUdy[iy]*(U[iy]-cs) - g*dHdy[iy]) 
#   P[iy]     = (-dGdy[iy]/(G[iy]*(U[iy]-cs)))
#   Q[iy]     = (F[iy]-(k**2))+(beta - d2Udy2[iy] - (HQ[iy]/G[iy])*dGdy[iy])/(U[iy]-cs)
 
                 
  return P, Q

In [36]:
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.0 : Define an ODE solver that will use the previous functions to solve the ODE
#------------------------------------------------------------------------------------------------------------------------------

def ODE_SOLVER_REES():
    import sys
    import math
    import cmath
    import numpy as np
    import matplotlib.pyplot as plt
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.1 : Set up the grid in y, allocate memory and define the constants
#------------------------------------------------------------------------------------------------------------------------------

    Levels = 100     # Number of levels
    Tolerance = 1e-7 # Tolerance for errors 
    
    y = np.zeros(Levels + 2)   # Allocate memory to y. Zeros is correct here because we give y proper values in the next loop.
    dy = 1.0/float(Levels)     # float/float = float
    len_y = len(y)             # len(y) = Levels + 2
    
    for n in range(Levels + 2):
        y[n] = (n-0.5)*dy  # Why not n*dy ? , why evaluate at half the grid, is this like a T-cell
        # This means that y varies from 0 to 1 in Levels + 2 steps (actually just less than 0 to just greater than 1)
       
    P      = np.zeros( len_y, dtype =  np.complex )     
    Q      = np.zeros( len_y, dtype =  np.complex )     
    f      = np.zeros( len_y, dtype =  np.complex )
    B      = np.zeros( len_y, dtype =  np.complex )
    H      = np.zeros( len_y, dtype =  np.complex )
    dHdy   = np.zeros( len_y, dtype =  np.complex )
    U      = np.zeros( len_y, dtype =  np.complex )
    dUdy   = np.zeros( len_y, dtype =  np.complex )
    HQ     = np.zeros( len_y, dtype =  np.complex )
    dHQdy  = np.zeros( len_y, dtype =  np.complex ) 
    D      = np.zeros( len_y, dtype =  np.complex )
    dDdy   = np.zeros( len_y, dtype =  np.complex )
    F      = np.zeros( len_y, dtype =  np.complex )
    dFdy   = np.zeros( len_y, dtype =  np.complex )
    d2Udy2 = np.zeros( len_y, dtype =  np.complex )
    beta   = 2.2*(10**-11)       
    g      = 0.0334              # Reduced gravity
    H0     = 60                  # Average thermocline depth (see Tanaka+Hibiya)
    A0     = -1.0                # Constant for n=0
    A2     = -1.0                # Constant for n=2
    c      = np.sqrt(g*H0) 
    C      = np.sqrt((2*beta/c)) 
    
    cs = np.zeros((1,1),dtype=np.complex) # Allocate memory for sigma that has an imaginary part
    c_0 = 0.0                             # Starting point for the wave speed
    d_c_inc = 0.001                       # Increment that we will iterate over
    d_c = d_c_inc + 1j * d_c_inc          # Increment that we add to the wave speed and iterate over
    k = 1.0                               # Prescribed choice of wavenumber 
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.2 : Find P and Q for c_0 (first choice of wave speed) and then integrate to find phi with boundary conditions
#------------------------------------------------------------------------------------------------------------------------------
    for iy in range ( len_y ) :
        f[iy]      = beta*y[iy]
        B[iy]      = (y[iy]**2)/4.0
        H[iy]      = A0 * np.exp(-B[iy])                                                     
        dHdy[iy]   = - (A0 / 2.0) * C * y[iy] * np.exp(-B[iy])                               
        U[iy]      = (g * A0 / c) * np.exp(-B[iy])                                           
        dUdy[iy]   = U[iy] * 0.5 * y[iy] * np.sqrt(C)
#       d2Udy2[iy] = ((g*A0*beta)/(2*c)) * ((y[iy]**2)-2)*np.exp(-B[iy])
        HQ[iy]     = beta * (1 / C) * y[iy] + (g * A2 / 2 * c) * C * y[iy] * np.exp(-B[iy]) 
        dHQdy[iy]  = beta*(1/C)*y[iy] + (g*A2 / 2*c)*C*y[iy]*np.exp(-B[iy]) 
        D[iy]      = (k**2) * ((U[iy] - c_0)**2) - f[iy] * HQ[iy]
        dDdy[iy]   = 2 * (k**2) * dUdy[iy] * (U[iy] - c_0) - beta * HQ[iy] - f[iy] * dHQdy[iy] #2 * (k_02) * dUdy[iy] * (U[iy] - c) - 2 * (beta**2) * y[iy] + beta * dUdy[iy] + beta * y[iy] * d2Udy2[iy]
        F[iy]      = D[iy] / g * H[iy]
        dFdy[iy]   = 1.0 / (g * H[iy]) * dDdy[iy] - D[iy] / g * H[iy] * (dHdy[iy])
        P[iy]      = (-1.0 / F[iy])*dFdy[iy]
        Q[iy]      = ((F[iy] - ((k)**2)) + ((f[iy] * dFdy[iy] / F[iy]) - beta) / (U[iy] - c_0))
    
    # Calculate the error in the solution with sigma_0
    P, Q = P_Q_Rees(y, c_0)    # Calculate P and Q for the initial choice of wavenumber

    PHI = np.zeros(Levels + 2, dtype = np.complex) # Allocate memory for phi
    PHI[0] = 1.0 # Sets the otherwise arbitary amplitude
    PHI[1] = 0.0 # Is this the correct boundary condition ?

    #Integrate up through the domain
    for n in range(1,Levels + 1): # Start at 1 otherwise Press[n-1] starts at Press[-1] # Also up to N_levs + 1 because y[n+1]=y[N_levs+2] which is the top of the boundary
        PHI[n+1] = INT_REES(PHI[n-1],PHI[n],P[n],Q[n],y[n-1],y[n],y[n+1])

    # Calculate the error at the upper boundary
    PHI_NU_BC = PHI[Levels]
    E_0 = PHI[Levels + 1] - PHI_NU_BC  # Phi 
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.3 : Iterate towards a solution for the ODE over all the wave speed increments
#------------------------------------------------------------------------------------------------------------------------------
    #Start iterating towards a solution (i.e. increment c forwards)
    c_1 = c_0 + d_c   # The next value for the wave speed is the initial prescribed value plus the increment
    Max_iter = 1      # Number of iterations
    for iter in range(Max_iter):

        P, Q = P_Q_Rees(y,c_1)   # Calculate P and Q for the new wave speed

        PHI = np.zeros(Levels + 2, dtype = np.complex) # Allocate memory for phi
        PHI[0] = 1.0 # Sets the otherwise arbitary amplitude
        PHI[1] = 0.0 # Boundary condition

        #Integrate up through the domain
        for n in range(1,Levels + 1):
            PHI[n+1] = INT_REES(PHI[n-1],PHI[n],P[n],Q[n],y[n-1],y[n],y[n+1]) # For c_1, find all the phi in the domain

        # Calculate the error at the upper boundary
        PHI_NU_BC = PHI[Levels]  
        E_1 = PHI[Levels + 1] - PHI_NU_BC
       
#------------------------------------------------------------------------------------------------------------------------------
# Section 3.4 : Check for convergence and prepare for the next loop
#------------------------------------------------------------------------------------------------------------------------------
        if abs(E_1) < Tolerance:    
            if i_kappa % Interval_kappa == 0:    # What is happening here ?
                cs[0,0] = c_1    #Sigma[i_soln,i_kappa] = 
                c_0 = c_1        # c_0 = c_1 so then on the next iteration, c_2 = c_1 + d_c and so forth
                break

            # Calculate the updated values of c
            gradient = (c_1 - c_0)/(E_1 - E_0)  # If the previous condition isn't satisfied then we calculate c using this method
            c_new = c_1 - gradient*E_1

            if abs(E_1) < abs(E_0): # If absolute value of the new error (c_1) is less than the old error (c_0) then,
                c_0 = c_1           # let c_0 = c_1, so that the next loop is actually for c_2
                E_0 = E_1           # Take the least error so that we can be sure that the solution is converging

            c_1 = c_new   # If the first if statement didn't hold then we get the new c from the gradient

            # If the solution fails to converge
            if abs(E_1) > Tolerance :
                break                
            
            # I am looping over all the increments of c, but don't these values need to be stored somewhere.
            # Is this not what cs is for ?
            
ODE_SOLVER_REES()

NameError: name 'ynm_1' is not defined

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
# Section 4.0 : Plots of the results
#------------------------------------------------------------------------------------------------------------------------------

plt.plot(k,k*cs.imag,'k',linewidth=2)   # but isn't k = 1.0 only, in my case ?
plt.title('Growth Rate')
plt.xlabel('Wavenumber k')
plt.ylabel('Growth Rate(kc_{i})')

In [31]:
print(cs)

NameError: name 'cs' is not defined

In [33]:
cs=np.zeros((1,1), dtype = np.complex)
print(cs)

[[0.+0.j]]
